# タクシー運賃の予測 - Part2:予測モデルの構築

Automated MLを利用し、タクシー運賃を予測する回帰モデルを構築します。
<br>
<br>
こちらは以下のチュートリアルをベースとしています。
<br>
https://docs.microsoft.com/ja-jp/azure/machine-learning/service/tutorial-auto-train-models

## 1．ワークスペースの構成

作成済みワークスペースから、ワークスペースのオブジェクトを取得します。

In [ ]:
import azureml.core
import pandas as pd
from azureml.core.workspace import Workspace
import logging
import os

ws = Workspace.from_config()

# 実験名、フォルダ名は必要に応じて変更してください
experiment_name = "automated-ml-regression"
project_folder = "./automated-ml-regression"

output = {}
output["SDK version"] = azureml.core.VERSION
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Project Directory"] = project_folder
pd.set_option("display.max_colwidth", -1)
pd.DataFrame(data=output, index=[""]).T

## 2．訓練データとテストデータの準備

データセットとして用意したDataflowオブジェクトを、デシリアライズします。

In [ ]:
import azureml.dataprep as dprep

file_path = os.path.join(os.getcwd(), "dflows.dprep")

dflow_prepared = dprep.Dataflow.open(file_path)
dflow_prepared.get_profile()

説明変数flow_Xと目的変数dflow_yを定義します。

In [ ]:
#dflow_X = dflow_prepared.keep_columns(["pickup_weekday","pickup_hour", "distance","passengers", "vendor"])
dflow_X = dflow_prepared.drop_columns("cost")
dflow_y = dflow_prepared.keep_columns("cost")

データセットをトレーニングデータとテストデータに分割します。

In [ ]:
from sklearn.model_selection import train_test_split

x_df = dflow_X.to_pandas_dataframe()
y_df = dflow_y.to_pandas_dataframe()

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)
y_train.values.flatten()

## 3．Automated MLによる予測モデルのトレーニング

Automated MLのパラメータを設定します。ここでは評価メトリックスをスピアマン相関としています。

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 10,
    "primary_metric" : "spearman_correlation",
    "preprocess" : True,
    "verbosity" : logging.INFO,
    "n_cross_validations": 5
}

automated_ml_config = AutoMLConfig(task = "regression",
                                  debug_log = "automated_ml_errors.log",
                                  path = project_folder,
                                  X = x_train.values,
                                  y = y_train.values.flatten(),
                                  **automl_settings)

予測モデルをトレーニングします。

In [ ]:
from azureml.core.experiment import Experiment

experiment = Experiment(ws, experiment_name)
local_run = experiment.submit(automated_ml_config, show_output=True)

## 4．トレーニング結果の確認

Jupyter Notebookを使っている場合、Jupyter Notebook Widgetを使って結果を視覚化できます。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(local_run).show()

Pythonのコードで、実行されたすべてのイテレーションのメトリックを取得します。

In [ ]:
children = list(local_run.get_children())
metricslist = {}

for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties["iteration"])] = metrics

pd.set_option("max_columns", 30)
rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

## 5．最適モデルの取得

実行されたイテレーションから最適なモデルを取得します。

In [ ]:
best_run, fitted_model = local_run.get_output()

print(best_run)
print(fitted_model)

## 6．最適モデルの正確性のテスト

最適モデルに対してテストデータを適用し、タクシー運賃を予測します。

In [ ]:
y_predict = fitted_model.predict(x_test.values)

print(y_predict[:10])

散布図を作成し、distance説明変数との相関を視覚化します。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(14, 10))
ax1 = fig.add_subplot(111)

distance_vals = [x[14] for x in x_test.values]
y_actual = y_test.values.flatten().tolist()

ax1.scatter(distance_vals[:100], y_predict[:100], s=18, c='b', marker="s", label='Predicted')
ax1.scatter(distance_vals[:100], y_actual[:100], s=18, c='r', marker="o", label='Actual')

ax1.set_xlabel('distance (mi)')
ax1.set_title('Predicted and Actual Cost/Distance')
ax1.set_ylabel('Cost ($)')

plt.legend(loc='upper left', prop={'size': 12})
plt.rcParams.update({'font.size': 14})
plt.show()